In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta
from pytz import timezone
import optuna

from hsml.schema import Schema
from hsml.model_schema import ModelSchema

from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline as PPLN
from sklearn.metrics import mean_absolute_error as MAE

import os
import sys
sys.path.append("../src/")
sys.path.append("../")

import config
import data
import data_split
import pipe
import paths
import inference
import featurestoreapi

In [4]:
currentdate = pd.to_datetime(datetime.utcnow()).floor("H") - timedelta(weeks=52)
print(currentdate)

2022-10-28 09:00:00


In [5]:
Features = inference.LoadBatchOfFeaturesFromStore(currentdate)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/104605
Connected. Call `.close()` to terminate connection gracefully.
Fetching data backwards from 2022-09-30 09:00:00 to 2022-10-28 08:00:00
Finished: Reading data from Hopsworks, using ArrowFlight (7.72s) 


In [6]:
Features

,rides_672_hours_before,rides_671_hours_before,rides_670_hours_before,rides_669_hours_before,rides_668_hours_before,rides_667_hours_before,rides_666_hours_before,rides_665_hours_before,rides_664_hours_before,rides_663_hours_before,...,rides_8_hours_before,rides_7_hours_before,rides_6_hours_before,rides_5_hours_before,rides_4_hours_before,rides_3_hours_before,rides_2_hours_before,rides_1_hours_before,pickup_hour,pickup_location_id
152,0.0,4.0,2.0,1.0,4.0,4.0,0.0,3.0,4.0,1.0,...,0.0,0.0,1.0,1.0,1.0,2.0,3.0,5.0,2022-10-28 09:00:00,1
118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-28 09:00:00,2
240,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-28 09:00:00,3
81,3.0,5.0,3.0,2.0,4.0,4.0,7.0,7.0,12.0,9.0,...,12.0,14.0,3.0,2.0,0.0,3.0,8.0,7.0,2022-10-28 09:00:00,4
34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-28 09:00:00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,32.0,34.0,32.0,34.0,48.0,63.0,57.0,57.0,45.0,43.0,...,8.0,1.0,2.0,7.0,4.0,3.0,13.0,13.0,2022-10-28 09:00:00,261
86,130.0,101.0,98.0,116.0,87.0,97.0,110.0,122.0,159.0,143.0,...,4.0,4.0,1.0,13.0,16.0,62.0,137.0,157.0,2022-10-28 09:00:00,262
223,130.0,123.0,119.0,132.0,121.0,133.0,153.0,144.0,184.0,238.0,...,38.0,28.0,18.0,14.0,21.0,80.0,132.0,128.0,2022-10-28 09:00:00,263
61,42.0,77.0,76.0,74.0,68.0,91.0,87.0,117.0,99.0,121.0,...,25.0,17.0,14.0,13.0,5.0,22.0,30.0,71.0,2022-10-28 09:00:00,264


In [7]:
Model = inference.LoadModelFromRegistry()
Predictions = inference.GetModelPredictions(Model, Features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/104605
Connected. Call `.close()` to terminate connection gracefully.

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [8]:
Predictions["pickup_hour"] = currentdate
Predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,1.0,2022-10-28 09:00:00
1,2,0.0,2022-10-28 09:00:00
2,3,0.0,2022-10-28 09:00:00
3,4,6.0,2022-10-28 09:00:00
4,5,0.0,2022-10-28 09:00:00
...,...,...,...
257,261,29.0,2022-10-28 09:00:00
258,262,135.0,2022-10-28 09:00:00
259,263,133.0,2022-10-28 09:00:00
260,264,80.0,2022-10-28 09:00:00


In [9]:
#Save the Predictions to the Feature Store for later usage

FeatureGroup = featurestoreapi.GetFeatureStore().get_or_create_feature_group(
    name = config.FeatureGroupModelPredictions,
    version = config.FeatureGroupModelPredictionsVersion,
    description = "Predictions Generated by our Production Model",
    primary_key = ["pickup_location_id", "pickup_hour"],
    event_time = "pickup_hour"
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/104605
Connected. Call `.close()` to terminate connection gracefully.


In [10]:
FeatureGroup.insert(Predictions, write_options = {"wait_for_job":False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/104605/fs/104524/fg/163167


Uploading Dataframe: 0.00% |          | Rows 0/262 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_updated_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/104605/jobs/named/model_predictions_feature_group_updated_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f9b8cabd8d0>, None)